# Automated ML

## Create an experiment in workspace

In [1]:
from azureml.core import Workspace, Experiment

#Define the workspace
ws = Workspace.from_config()

#Create an experiment
exp = Experiment(workspace=ws, name="AutoML")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
run = exp.start_logging()

Workspace name: quick-starts-ws-135681
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-135681


## Create a compute cluster

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

#Create a new cluster to run the experiment using vm_size of STANDARD_D2_V2 and max_nodes of 4. 
cpu_cluster_name = "cpucluster"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found an existing cluster.\n")
except ComputeTargetException:
    print("Creating a new cluster.\n")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion()

#If the cluster exist already for the current cluster, print it's detailed status using get_status(). If not, create and print.
print("Cluster details: ", cpu_cluster.get_status().serialize())

Found an existing cluster.

Cluster details:  {'currentNodeCount': 4, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 4, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2021-01-22T09:18:04.039000+00:00', 'errors': None, 'creationTime': '2021-01-22T08:47:59.411726+00:00', 'modifiedTime': '2021-01-22T08:48:14.900351+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset overview

This project demonstrates the usage of different Machine Learning Algorithms on the Kaggle's Titanic dataset. We are performing classification in this case. The titanic dataset consists of features related to a passenger and the response is if a passenger survived the titanic disaster or not (Survived-1/0).

In [3]:
import pandas as pd

#Loading the titanic dataset
df = pd.read_csv('https://raw.githubusercontent.com/parvatijay2901/Machine-Learning-with-the-Titanic-dataset-on-Azure/main/Training_data.csv')

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Analyse the data, transform and split the dataset to train and test sets. 

In [4]:
from train import clean_data
from azureml.core import Dataset
from sklearn.model_selection import train_test_split
from azureml.data.dataset_factory import TabularDatasetFactory

#Use the clean_data function to clean your dataset.
x, y = clean_data(df)
x.head()

#Split the dataset into train and test dataset. Combine x_train and y_train. 
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
df_train = pd.concat([x_train,y_train], axis=1)
df_test = pd.concat([x_test,y_test], axis=1)

#Convert x_train and y_train (Which are in pandas DataFrame format) to TabularDataset format.
try:
    os.makedirs('./data', exist_ok=True)
except OSError as error:
    print('New directory cannot be created')
    
path_train = 'data/train.csv'
path_test = 'data/test.csv'
df_train.to_csv(path_train)
df_test.to_csv(path_test)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

train_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/train.csv'))])
test_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/test.csv'))])
print("Successfully converted the dataset to TabularDataset format.")

Uploading an estimated of 2 files
Uploading data/test.csv
Uploaded data/test.csv, 1 files out of an estimated total of 2
Uploading data/train.csv
Uploaded data/train.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Successfully converted the dataset to TabularDataset format.


## AutoML Configuration

AutoML rapidly iterates over many combinations of algorithms and hyperparameters, and find out the best classification model based on the primary metric - Accuracy. In our case the experiment gets timed out after 30mins. I have also enabled onnx compatible models.  


In [5]:
from azureml.train.automl import AutoMLConfig

#Set parameters for AutoMLConfig
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="Survived",
    n_cross_validations=5,
    enable_onnx_compatible_models=True,
    compute_target=cpu_cluster)

## Run Details

Automated machine learning supports ensemble models. Ensemble learning improves machine learning results and predictive performance by combining multiple models as opposed to using single models (Example, Voting ensemble, stack ensemble). Here we are performing AutoML run on a remote compute cluster. 

In [6]:
#Run AutoML on a remote compute target
remote_run = exp.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpucluster with default configuration
Running on remote compute: cpucluster
Parent Run ID: AutoML_025f2b39-b2d8-4bca-bfe4-eaabcae87cef

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: ht

In [7]:
from azureml.widgets import RunDetails

#Launch the widget to view the progress and results
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Retrieve and save the best AutoML model

In [8]:
#Retrieving the best model
best_run_AutoML, best_model_AutoML = remote_run.get_output()
best_run_metrics_AutoML = best_run_AutoML.get_metrics()
print("Best run Id: ",best_run_AutoML.id)
print("Accuracy: ", best_run_metrics_AutoML['accuracy'])
print("Other details: ")
print("Fitted model:",best_model_AutoML)

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


Best run Id:  AutoML_025f2b39-b2d8-4bca-bfe4-eaabcae87cef_26
Accuracy:  0.8118191667487442
Other details: 
Fitted model: Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('MaxAbsScaler', MaxAbsScaler(copy...
                                    learning_rate=0.08947473684210526,
                                    max_bin=310, max_depth=3,
                                    min_child_samples=25, min_child_weight=6,
                                    min_split_gain=0.473684

In [9]:
best_run_AutoML.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":1}',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

In [10]:
import joblib
from azureml.core.model import Model

#Save the best model
os.makedirs('results', exist_ok=True)
joblib.dump(best_model_AutoML, filename="results/automl_model.pkl")
model = remote_run.register_model(model_name=best_run_AutoML.properties['model_name'], description='Best AutoML model')
print("Model saved successfully")

Model saved successfully


## Retrieve and save the best ONNX model

In [11]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

#Retrieve and save the best model
best_run, onnx_mdl = remote_run.get_output(return_onnx_model=True)
onnx_fl_path = "results/best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

## Predict with the ONNX model

In [12]:
import sys
import json
from azureml.automl.core.onnx_convert import OnnxConvertConstants
from azureml.train.automl import constants

if sys.version_info < OnnxConvertConstants.OnnxIncompatiblePythonVersion:
    python_version_compatible = True
else:
    python_version_compatible = False

import onnxruntime
from azureml.automl.runtime.onnx_convert import OnnxInferenceHelper

def get_onnx_res(run):
    res_path = 'onnx_resource.json'
    run.download_file(name=constants.MODEL_RESOURCE_PATH_ONNX, output_file_path=res_path)
    with open(res_path) as f:
        onnx_res = json.load(f)
    return onnx_res

if python_version_compatible:
    test_df = test_data.to_pandas_dataframe()
    mdl_bytes = onnx_mdl.SerializeToString()
    onnx_res = get_onnx_res(best_run)

    onnxrt_helper = OnnxInferenceHelper(mdl_bytes, onnx_res)
    pred_onnx, pred_prob_onnx = onnxrt_helper.predict(test_df)

    print(pred_onnx)
    print(pred_prob_onnx)
else:
    print('Please use Python version 3.6 or 3.7 to run the inference helper.')

[0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0
 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0
 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 0 0 1 1
 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 1 0 1 1 0]
[[0.8456826  0.15431735]
 [0.71387124 0.28612882]
 [0.83192855 0.16807145]
 [0.08670013 0.91329986]
 [0.6583085  0.34169152]
 [0.7922295  0.20777056]
 [0.08389928 0.9161007 ]
 [0.87143546 0.12856457]
 [0.7731743  0.22682568]
 [0.73973024 0.2602697 ]
 [0.83730286 0.16269714]
 [0.8089965  0.19100347]
 [0.641677   0.35832292]
 [0.08692507 0.913075  ]
 [0.79855335 0.20144665]
 [0.84633476 0.15366527]
 [0.7051767  0.2948233 ]
 [0.895187   0.10481305]
 [0.8739994  0.1260006 ]
 [0.39074755 0.60925245]
 [0.6307692  0.36923084]
 [0.59139484 0.40860516]
 [0.75056654 0.24943347]
 [0.8027035  0.19729656]
 [0.77863497 0.22136502]
 [0.6421248

## Deploy the best model 

In [20]:
env = best_run_AutoML.get_environment()
entry_script='score.py'
best_run_AutoML.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)

In [21]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

#Deploy the model as a Web Service on Azure Container Instance
inference_config = InferenceConfig(entry_script = entry_script, environment = env)
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled=True, enable_app_insights=True)
service = Model.deploy(ws, 'titanic-api', [model], inference_config, deployment_config)
service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [22]:
#Gets logs from a deployed web service.
print(service.get_logs())

2021-01-22T11:36:27,549761760+00:00 - iot-server/run 
2021-01-22T11:36:27,549758860+00:00 - gunicorn/run 
2021-01-22T11:36:27,550309056+00:00 - rsyslog/run 
2021-01-22T11:36:27,552448741+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [16]:
print("Deployment state: " + service.state)
print("Scoring URI: " + service.scoring_uri)
print("Authetication Key: " + service.get_keys()[0])
print("Swagger URI: " + service.swagger_uri)

Deployment state: Healthy
Scoring URI: http://36c3231c-988a-4252-8ab3-8798c041165e.southcentralus.azurecontainer.io/score
Authetication Key: RpFLxIGW4uMcLZlKU0iy4JcOa04YVd8c
Swagger URI: http://36c3231c-988a-4252-8ab3-8798c041165e.southcentralus.azurecontainer.io/swagger.json


In [18]:
#Consume model endpoint
%run endpoint.py

{"result": [0, 1]}


In [19]:
#Delete the web service
service.delete()